In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
#dataset_5_ms = pd.read_csv('Datasets/dataset_halfSecondWindow.csv', index_col=[0])
dataset = pd.read_csv('Datasets/dataset_5SecondWindow.csv', index_col=[0])

In [4]:
dataset.head()

,id,time,activityrecognition#0,activityrecognition#1,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.accelerometer#std,android.sensor.game_rotation_vector#mean,android.sensor.game_rotation_vector#min,android.sensor.game_rotation_vector#max,android.sensor.game_rotation_vector#std,android.sensor.gravity#mean,android.sensor.gravity#min,android.sensor.gravity#max,android.sensor.gravity#std,android.sensor.gyroscope#mean,android.sensor.gyroscope#min,android.sensor.gyroscope#max,android.sensor.gyroscope#std,android.sensor.gyroscope_uncalibrated#mean,android.sensor.gyroscope_uncalibrated#min,android.sensor.gyroscope_uncalibrated#max,android.sensor.gyroscope_uncalibrated#std,android.sensor.light#mean,android.sensor.light#min,android.sensor.light#max,android.sensor.light#std,android.sensor.linear_acceleration#mean,android.sensor.linear_acceleration#min,android.sensor.linear_acceleration#max,android.sensor.linear_acceleration#std,android.sensor.magnetic_field#mean,android.sensor.magnetic_field#min,android.sensor.magnetic_field#max,android.sensor.magnetic_field#std,android.sensor.magnetic_field_uncalibrated#mean,android.sensor.magnetic_field_uncalibrated#min,android.sensor.magnetic_field_uncalibrated#max,android.sensor.magnetic_field_uncalibrated#std,android.sensor.orientation#mean,android.sensor.orientation#min,android.sensor.orientation#max,android.sensor.orientation#std,android.sensor.pressure#mean,android.sensor.pressure#min,android.sensor.pressure#max,android.sensor.pressure#std,android.sensor.proximity#mean,android.sensor.proximity#min,android.sensor.proximity#max,android.sensor.proximity#std,android.sensor.rotation_vector#mean,android.sensor.rotation_vector#min,android.sensor.rotation_vector#max,android.sensor.rotation_vector#std,android.sensor.step_counter#mean,android.sensor.step_counter#min,android.sensor.step_counter#max,android.sensor.step_counter#std,sound#mean,sound#min,sound#max,sound#std,speed#mean,speed#min,speed#max,speed#std,target,user
0,16170,78.0,NaN,100.0,9.811476,9.758895,9.849411,0.014626,0.029340,0.029014,0.029526,0.000119,9.806650,9.806649,9.806651,4.780692e-07,0.001651,0.000000,0.003533,0.000737,0.016221,0.014172,0.018695,0.000982,0.0,0.0,0.0,0.000000,0.020978,0.002495,0.052410,0.011045,57.099638,56.690387,57.575950,0.177549,51.363566,51.199707,51.539208,0.080899,354.286933,353.598335,354.942707,0.245676,1004.090261,1004.05540,1004.12790,0.017416,8.0,8.0,8.0,NaN,0.050413,0.044777,0.056351,0.002109,28966.0,28966.0,28966.0,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
1,15871,145.0,NaN,100.0,9.939207,7.707437,17.146631,1.775944,0.999925,0.999903,0.999946,0.000030,9.806624,9.806624,9.806624,6.474977e-07,0.036326,0.011669,0.059388,0.020290,0.039023,0.014132,0.085494,0.018629,0.0,0.0,0.0,NaN,0.879220,0.641117,1.185810,0.278730,29.351288,28.172505,30.386017,0.921547,82.767760,82.409890,83.125630,0.506105,332.695577,330.461054,339.108607,1.705816,1008.274660,1008.27466,1008.27466,NaN,NaN,NaN,NaN,NaN,0.999981,0.999963,0.999999,0.000026,NaN,NaN,NaN,NaN,89.200210,89.065143,89.335277,0.191013,16.539349,16.539349,16.539349,0.628595,Car,U12
2,16811,150.0,NaN,100.0,9.827178,9.804817,9.849262,0.011199,0.665215,0.665213,0.665218,0.000004,9.806650,9.806650,9.806650,2.821568e-07,0.001525,0.000000,0.002609,0.000713,0.016302,0.014645,0.017374,0.000876,2.0,2.0,2.0,0.707107,0.034110,0.025709,0.042511,0.011881,29.904927,29.741722,30.038309,0.130851,55.695716,55.501802,55.868506,0.128945,74.752653,73.223493,75.268574,0.413704,1005.977540,1005.97754,1005.97754,NaN,8.0,8.0,8.0,NaN,0.610456,0.610456,0.610456,0.011312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
3,15831,105.0,NaN,77.0,9.673039,7.659674,12.304298,0.862553,0.996221,0.993781,0.998661,0.003451,9.806638,9.806638,9.806638,2.120564e-06,0.036440,0.020463,0.052512,0.010553,0.050759,0.024732,0.081887,0.018037,0.0,0.0,0.0,NaN,1.488361,0.575910,2.415879,0.920077,37.

In [5]:
dataset.columns

Index(['id', 'time', 'activityrecognition#0', 'activityrecognition#1',
       'android.sensor.accelerometer#mean', 'android.sensor.accelerometer#min',
       'android.sensor.accelerometer#max', 'android.sensor.accelerometer#std',
       'android.sensor.game_rotation_vector#mean',
       'android.sensor.game_rotation_vector#min',
       'android.sensor.game_rotation_vector#max',
       'android.sensor.game_rotation_vector#std',
       'android.sensor.gravity#mean', 'android.sensor.gravity#min',
       'android.sensor.gravity#max', 'android.sensor.gravity#std',
       'android.sensor.gyroscope#mean', 'android.sensor.gyroscope#min',
       'android.sensor.gyroscope#max', 'android.sensor.gyroscope#std',
       'android.sensor.gyroscope_uncalibrated#mean',
       'android.sensor.gyroscope_uncalibrated#min',
       'android.sensor.gyroscope_uncalibrated#max',
       'android.sensor.gyroscope_uncalibrated#std',
       'android.sensor.light#mean', 'android.sensor.light#min',
       'android.se

id
time
activityrecognition # 2
android.sensor.accelerometer #mmms //Measures the acceleration force in m/s2 that is applied to a device on all three physical axes (x, y, and z), including the force of gravityacceleration force
android.sensor.game_rotation_vector # mmms //orientation of the device
android.sensor.gravity #mmms //Measures the force of gravity in m/s2 that is applied to a device on all three physical axes (x, y, z)
android.sensor.gyroscope #mmms //Measures a device's rate of rotation in rad/s around each of the three physical axes (x, y, and z)
android.sensor.light #mmms //Measures the ambient light level (illumination) in lx
android.sensor.linear_acceleration #mmms // Measures the acceleration force in m/s2 that is applied to a device on all three physical axes (x, y, and z), excluding the force of gravity
android.sensor.magnetic_field #mmms //Measures the ambient geomagnetic field for all three physical axes (x, y, z) in μT
android.sensor.magnetic_field_uncalibrated #mmms
android.sensor.orientation #mmms
android.sensor.pressure #mmms //Measures the ambient air pressure in hPa or mbar.
android.sensor.proximity #mmms //Measures the proximity of an object in cm relative to the view screen of a device. This sensor is typically used to determine whether a handset is being held up to a person's ear.
android.sensor.rotation_vector #mmms //Measures the orientation of a device by providing the three elements of the device's rotation vector.
android.sensor.step_counter #mmms
sound #mmms
speed #mmms
target
user

targets : walking, car, still, train and bus. 
26% of data is annoted as walking, 25% as driving a car, 24% as standing still, 20% as being on train, and 5% as being on bus. 

In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5893 entries, 0 to 5892
Data columns (total 70 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   id                                               5893 non-null   int64  
 1   time                                             5893 non-null   float64
 2   activityrecognition#0                            0 non-null      float64
 3   activityrecognition#1                            5893 non-null   float64
 4   android.sensor.accelerometer#mean                5893 non-null   float64
 5   android.sensor.accelerometer#min                 5893 non-null   float64
 6   android.sensor.accelerometer#max                 5893 non-null   float64
 7   android.sensor.accelerometer#std                 5893 non-null   float64
 8   android.sensor.game_rotation_vector#mean         5094 non-null   float64
 9   android.sensor.game_rotation_v

In [7]:
dataset.describe()

,id,time,activityrecognition#0,activityrecognition#1,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.accelerometer#std,android.sensor.game_rotation_vector#mean,android.sensor.game_rotation_vector#min,android.sensor.game_rotation_vector#max,android.sensor.game_rotation_vector#std,android.sensor.gravity#mean,android.sensor.gravity#min,android.sensor.gravity#max,android.sensor.gravity#std,android.sensor.gyroscope#mean,android.sensor.gyroscope#min,android.sensor.gyroscope#max,android.sensor.gyroscope#std,android.sensor.gyroscope_uncalibrated#mean,android.sensor.gyroscope_uncalibrated#min,android.sensor.gyroscope_uncalibrated#max,android.sensor.gyroscope_uncalibrated#std,android.sensor.light#mean,android.sensor.light#min,android.sensor.light#max,android.sensor.light#std,android.sensor.linear_acceleration#mean,android.sensor.linear_acceleration#min,android.sensor.linear_acceleration#max,android.sensor.linear_acceleration#std,android.sensor.magnetic_field#mean,android.sensor.magnetic_field#min,android.sensor.magnetic_field#max,android.sensor.magnetic_field#std,android.sensor.magnetic_field_uncalibrated#mean,android.sensor.magnetic_field_uncalibrated#min,android.sensor.magnetic_field_uncalibrated#max,android.sensor.magnetic_field_uncalibrated#std,android.sensor.orientation#mean,android.sensor.orientation#min,android.sensor.orientation#max,android.sensor.orientation#std,android.sensor.pressure#mean,android.sensor.pressure#min,android.sensor.pressure#max,android.sensor.pressure#std,android.sensor.proximity#mean,android.sensor.proximity#min,android.sensor.proximity#max,android.sensor.proximity#std,android.sensor.rotation_vector#mean,android.sensor.rotation_vector#min,android.sensor.rotation_vector#max,android.sensor.rotation_vector#std,android.sensor.step_counter#mean,android.sensor.step_counter#min,android.sensor.step_counter#max,android.sensor.step_counter#std,sound#mean,sound#min,sound#max,sound#std,speed#mean,speed#min,speed#max,speed#std
count,5893.000000,5893.000000,0.0,5893.000000,5893.000000,5893.000000,5893.000000,5893.000000,5094.000000,5094.000000,5094.000000,5093.000000,5396.000000,5396.000000,5396.000000,5.356000e+03,5173.000000,5173.000000,5173.000000,5172.000000,5094.000000,5094.000000,5094.000000,5094.000000,4381.000000,4381.000000,4381.000000,2124.000000,5399.000000,5399.000000,5399.000000,5370.000000,5400.000000,5400.000000,5400.000000,5383.000000,5094.000000,5094.000000,5094.000000,5087.000000,5396.000000,5396.000000,5396.000000,5373.000000,1918.000000,1918.000000,1918.000000,1128.000000,1119.000000,1119.000000,1119.000000,283.000000,5172.000000,5172.000000,5172.000000,5163.000000,1487.000000,1487.000000,1487.000000,232.000000,4899.000000,4899.000000,4899.000000,4062.000000,3520.000000,3520.000000,3520.000000,1318.000000
mean,10685.407263,117.731716,NaN,87.718310,10.007976,8.426262,12.247691,0.982987,0.759280,0.743872,0.771629,0.011876,9.806561,9.806530,9.806592,1.220395e-04,0.312536,0.089494,0.735042,0.189903,0.312807,0.125427,0.614961,0.174279,288.825545,254.140861,331.698065,85.357327,1.315971,0.863086,2.157592,0.602771,56.212508,54.267805,58.207109,1.772942,138.409066,136.955896,139.907248,2.038359,212.629637,199.930476,224.731494,12.488254,1016.203139,1016.189143,1016.218816,0.023097,6.117128,5.174263,6.797140,4.081700,0.765932,0.750273,0.779000,0.012977,10112.523523,10112.205783,10112.769334,2.959475,78.724855,78.451658,78.997213,1.913065,8.235254,8.216747,8.253528,0.236959
std,6921.229833,126.474151,NaN,24.562728,0.655949,1.783007,5.178436,1.505615,0.251174,0.255232,0.251401,0.026647,0.001610,0.001709,0.001584,7.681037e-04,0.554585,0.206327,1.356240,0.342901,0.537925,0.235223,1.112451,0.322397,1761.573894,1361.427348,2379.942459,1052.057842,2.065444,1.493150,5.058223,1.136752,38.210015,37.902702,38.681821,2.938644,71.034674,70.764992,71.910605,5.423627,95.253318,98.160410,101.113269,31.517931,14.142150,14.146752,14.137338,0.081745,15.141369,15.34

In [8]:
dataset.isnull().sum()

id                                                    0
time                                                  0
activityrecognition#0                              5893
activityrecognition#1                                 0
android.sensor.accelerometer#mean                     0
android.sensor.accelerometer#min                      0
android.sensor.accelerometer#max                      0
android.sensor.accelerometer#std                      0
android.sensor.game_rotation_vector#mean            799
android.sensor.game_rotation_vector#min             799
android.sensor.game_rotation_vector#max             799
android.sensor.game_rotation_vector#std             800
android.sensor.gravity#mean                         497
android.sensor.gravity#min                          497
android.sensor.gravity#max                          497
android.sensor.gravity#std                          537
android.sensor.gyroscope#mean                       720
android.sensor.gyroscope#min                    

In [9]:
# drop std
to_drop = [c for c in dataset.columns if '#std' in c]
dataset.drop(to_drop, axis=1, inplace=True)
dataset.head()

,id,time,activityrecognition#0,activityrecognition#1,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.game_rotation_vector#mean,android.sensor.game_rotation_vector#min,android.sensor.game_rotation_vector#max,android.sensor.gravity#mean,android.sensor.gravity#min,android.sensor.gravity#max,android.sensor.gyroscope#mean,android.sensor.gyroscope#min,android.sensor.gyroscope#max,android.sensor.gyroscope_uncalibrated#mean,android.sensor.gyroscope_uncalibrated#min,android.sensor.gyroscope_uncalibrated#max,android.sensor.light#mean,android.sensor.light#min,android.sensor.light#max,android.sensor.linear_acceleration#mean,android.sensor.linear_acceleration#min,android.sensor.linear_acceleration#max,android.sensor.magnetic_field#mean,android.sensor.magnetic_field#min,android.sensor.magnetic_field#max,android.sensor.magnetic_field_uncalibrated#mean,android.sensor.magnetic_field_uncalibrated#min,android.sensor.magnetic_field_uncalibrated#max,android.sensor.orientation#mean,android.sensor.orientation#min,android.sensor.orientation#max,android.sensor.pressure#mean,android.sensor.pressure#min,android.sensor.pressure#max,android.sensor.proximity#mean,android.sensor.proximity#min,android.sensor.proximity#max,android.sensor.rotation_vector#mean,android.sensor.rotation_vector#min,android.sensor.rotation_vector#max,android.sensor.step_counter#mean,android.sensor.step_counter#min,android.sensor.step_counter#max,sound#mean,sound#min,sound#max,speed#mean,speed#min,speed#max,target,user
0,16170,78.0,NaN,100.0,9.811476,9.758895,9.849411,0.029340,0.029014,0.029526,9.806650,9.806649,9.806651,0.001651,0.000000,0.003533,0.016221,0.014172,0.018695,0.0,0.0,0.0,0.020978,0.002495,0.052410,57.099638,56.690387,57.575950,51.363566,51.199707,51.539208,354.286933,353.598335,354.942707,1004.090261,1004.05540,1004.12790,8.0,8.0,8.0,0.050413,0.044777,0.056351,28966.0,28966.0,28966.0,NaN,NaN,NaN,0.000000,0.000000,0.000000,Still,U12
1,15871,145.0,NaN,100.0,9.939207,7.707437,17.146631,0.999925,0.999903,0.999946,9.806624,9.806624,9.806624,0.036326,0.011669,0.059388,0.039023,0.014132,0.085494,0.0,0.0,0.0,0.879220,0.641117,1.185810,29.351288,28.172505,30.386017,82.767760,82.409890,83.125630,332.695577,330.461054,339.108607,1008.274660,1008.27466,1008.27466,NaN,NaN,NaN,0.999981,0.999963,0.999999,NaN,NaN,NaN,89.200210,89.065143,89.335277,16.539349,16.539349,16.539349,Car,U12
2,16811,150.0,NaN,100.0,9.827178,9.804817,9.849262,0.665215,0.665213,0.665218,9.806650,9.806650,9.806650,0.001525,0.000000,0.002609,0.016302,0.014645,0.017374,2.0,2.0,2.0,0.034110,0.025709,0.042511,29.904927,29.741722,30.038309,55.695716,55.501802,55.868506,74.752653,73.223493,75.268574,1005.977540,1005.97754,1005.97754,8.0,8.0,8.0,0.610456,0.610456,0.610456,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,Still,U12
3,15831,105.0,NaN,77.0,9.673039,7.659674,12.304298,0.996221,0.993781,0.998661,9.806638,9.806638,9.806638,0.036440,0.020463,0.052512,0.050759,0.024732,0.081887,0.0,0.0,0.0,1.488361,0.575910,2.415879,37.836471,37.525725,38.147217,95.743821,95.664309,95.823332,373.556039,359.897250,397.071741,1006.950440,1006.95044,1006.95044,NaN,NaN,NaN,0.998112,0.998112,0.998112,NaN,NaN,NaN,87.470377,87.470377,87.470377,17.739895,17.739895,17.739895,Car,U12
4,876,77.0,NaN,100.0,9.993466,8.965621,10.891645,0.563792,0.521799,0.672664,9.806650,9.806650,9.806651,0.183202,0.020667,0.380875,0.265652,0.141208,0.313020,30.0,30.0,30.0,1.658308,0.379803,2.795995,78.375718,76.543261,79.273739,157.429218,156.795909,158.062526,193.201458,35.864972,360.560468,NaN,NaN,NaN,NaN,NaN,NaN,0.322242,0.308055,0.378193,NaN,NaN,NaN,89.770732,89.770732,89.770732,9.000000,9.000000,9.000000,Car,U1


In [10]:
# replace anroid.sensor & #
dataset.columns = dataset.columns.str.replace('android.sensor.','').str.replace('#','_')
dataset.head()

<ipython-input-10-e92dca201964>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  dataset.columns = dataset.columns.str.replace('android.sensor.','').str.replace('#','_')


,id,time,activityrecognition_0,activityrecognition_1,accelerometer_mean,accelerometer_min,accelerometer_max,game_rotation_vector_mean,game_rotation_vector_min,game_rotation_vector_max,gravity_mean,gravity_min,gravity_max,gyroscope_mean,gyroscope_min,gyroscope_max,gyroscope_uncalibrated_mean,gyroscope_uncalibrated_min,gyroscope_uncalibrated_max,light_mean,light_min,light_max,linear_acceleration_mean,linear_acceleration_min,linear_acceleration_max,magnetic_field_mean,magnetic_field_min,magnetic_field_max,magnetic_field_uncalibrated_mean,magnetic_field_uncalibrated_min,magnetic_field_uncalibrated_max,orientation_mean,orientation_min,orientation_max,pressure_mean,pressure_min,pressure_max,proximity_mean,proximity_min,proximity_max,rotation_vector_mean,rotation_vector_min,rotation_vector_max,step_counter_mean,step_counter_min,step_counter_max,sound_mean,sound_min,sound_max,speed_mean,speed_min,speed_max,target,user
0,16170,78.0,NaN,100.0,9.811476,9.758895,9.849411,0.029340,0.029014,0.029526,9.806650,9.806649,9.806651,0.001651,0.000000,0.003533,0.016221,0.014172,0.018695,0.0,0.0,0.0,0.020978,0.002495,0.052410,57.099638,56.690387,57.575950,51.363566,51.199707,51.539208,354.286933,353.598335,354.942707,1004.090261,1004.05540,1004.12790,8.0,8.0,8.0,0.050413,0.044777,0.056351,28966.0,28966.0,28966.0,NaN,NaN,NaN,0.000000,0.000000,0.000000,Still,U12
1,15871,145.0,NaN,100.0,9.939207,7.707437,17.146631,0.999925,0.999903,0.999946,9.806624,9.806624,9.806624,0.036326,0.011669,0.059388,0.039023,0.014132,0.085494,0.0,0.0,0.0,0.879220,0.641117,1.185810,29.351288,28.172505,30.386017,82.767760,82.409890,83.125630,332.695577,330.461054,339.108607,1008.274660,1008.27466,1008.27466,NaN,NaN,NaN,0.999981,0.999963,0.999999,NaN,NaN,NaN,89.200210,89.065143,89.335277,16.539349,16.539349,16.539349,Car,U12
2,16811,150.0,NaN,100.0,9.827178,9.804817,9.849262,0.665215,0.665213,0.665218,9.806650,9.806650,9.806650,0.001525,0.000000,0.002609,0.016302,0.014645,0.017374,2.0,2.0,2.0,0.034110,0.025709,0.042511,29.904927,29.741722,30.038309,55.695716,55.501802,55.868506,74.752653,73.223493,75.268574,1005.977540,1005.97754,1005.97754,8.0,8.0,8.0,0.610456,0.610456,0.610456,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,Still,U12
3,15831,105.0,NaN,77.0,9.673039,7.659674,12.304298,0.996221,0.993781,0.998661,9.806638,9.806638,9.806638,0.036440,0.020463,0.052512,0.050759,0.024732,0.081887,0.0,0.0,0.0,1.488361,0.575910,2.415879,37.836471,37.525725,38.147217,95.743821,95.664309,95.823332,373.556039,359.897250,397.071741,1006.950440,1006.95044,1006.95044,NaN,NaN,NaN,0.998112,0.998112,0.998112,NaN,NaN,NaN,87.470377,87.470377,87.470377,17.739895,17.739895,17.739895,Car,U12
4,876,77.0,NaN,100.0,9.993466,8.965621,10.891645,0.563792,0.521799,0.672664,9.806650,9.806650,9.806651,0.183202,0.020667,0.380875,0.265652,0.141208,0.313020,30.0,30.0,30.0,1.658308,0.379803,2.795995,78.375718,76.543261,79.273739,157.429218,156.795909,158.062526,193.201458,35.864972,360.560468,NaN,NaN,NaN,NaN,NaN,NaN,0.322242,0.308055,0.378193,NaN,NaN,NaN,89.770732,89.770732,89.770732,9.000000,9.000000,9.000000,Car,U1


In [11]:
dataset = dataset[(['accelerometer_mean',
                  'accelerometer_min',
                  'accelerometer_max',
                  'gyroscope_mean',
                  'linear_acceleration_mean',
                  'orientation_mean',
                  'pressure_mean',
                  'rotation_vector_mean',
                  'step_counter_mean',
                  'sound_mean',
                  'user',
                  'target'
                 ])]

In [12]:
dataset.head()

,accelerometer_mean,accelerometer_min,accelerometer_max,gyroscope_mean,linear_acceleration_mean,orientation_mean,pressure_mean,rotation_vector_mean,step_counter_mean,sound_mean,user,target
0,9.811476,9.758895,9.849411,0.001651,0.020978,354.286933,1004.090261,0.050413,28966.0,NaN,U12,Still
1,9.939207,7.707437,17.146631,0.036326,0.879220,332.695577,1008.274660,0.999981,NaN,89.200210,U12,Car
2,9.827178,9.804817,9.849262,0.001525,0.034110,74.752653,1005.977540,0.610456,NaN,NaN,U12,Still
3,9.673039,7.659674,12.304298,0.036440,1.488361,373.556039,1006.950440,0.998112,NaN,87.470377,U12,Car
4,9.993466,8.965621,10.891645,0.183202,1.658308,193.201458,NaN,0.322242,NaN,89.770732,U1,Car


In [13]:
#dataset['user']=pd.factorize(dataset['user'].tolist())[0] #ordinal encode user

In [14]:
# dataset = dataset.drop(['user'], axis=1)

In [15]:
subset = dataset[0:100]
subset

,accelerometer_mean,accelerometer_min,accelerometer_max,gyroscope_mean,linear_acceleration_mean,orientation_mean,pressure_mean,rotation_vector_mean,step_counter_mean,sound_mean,user,target
0,9.811476,9.758895,9.849411,0.001651,0.020978,354.286933,1004.090261,0.050413,28966.00,NaN,U12,Still
1,9.939207,7.707437,17.146631,0.036326,0.879220,332.695577,1008.274660,0.999981,NaN,89.200210,U12,Car
2,9.827178,9.804817,9.849262,0.001525,0.034110,74.752653,1005.977540,0.610456,NaN,NaN,U12,Still
3,9.673039,7.659674,12.304298,0.036440,1.488361,373.556039,1006.950440,0.998112,NaN,87.470377,U12,Car
4,9.993466,8.965621,10.891645,0.183202,1.658308,193.201458,NaN,0.322242,NaN,89.770732,U1,Car
5,9.330482,9.149029,9.520044,NaN,NaN,NaN,NaN,NaN,NaN,57.361127,U8,Car
6,9.789771,9.758751,9.816197,0.004100,0.027441,336.891507,1036.933200,0.201198,NaN,65.158369,U7,Train
7,9.906011,6.431646,14.202603,0.140902,1.003520,253.102379,1007.783480,0.929187,12992.25,NaN,U12,Bus
8,9.639711,8.996134,10.027802,0.041554,0.278997,131.294098,NaN,0.780795,NaN,89.808441,U1,Bus
9,9.602109,9.014786,10.068528,0.037451,0.255172,151.073183,NaN,0.968082,NaN,89.815738,U1,Train


In [17]:
subset_test = subset.loc[subset['user'] == 'U12']


In [18]:
subset_test

,accelerometer_mean,accelerometer_min,accelerometer_max,gyroscope_mean,linear_acceleration_mean,orientation_mean,pressure_mean,rotation_vector_mean,step_counter_mean,sound_mean,user,target
0,9.811476,9.758895,9.849411,0.001651,0.020978,354.286933,1004.090261,0.050413,28966.00,NaN,U12,Still
1,9.939207,7.707437,17.146631,0.036326,0.879220,332.695577,1008.274660,0.999981,NaN,89.200210,U12,Car
2,9.827178,9.804817,9.849262,0.001525,0.034110,74.752653,1005.977540,0.610456,NaN,NaN,U12,Still
3,9.673039,7.659674,12.304298,0.036440,1.488361,373.556039,1006.950440,0.998112,NaN,87.470377,U12,Car
7,9.906011,6.431646,14.202603,0.140902,1.003520,253.102379,1007.783480,0.929187,12992.25,NaN,U12,Bus
18,10.098494,4.184305,20.965369,1.936651,6.589222,178.274932,NaN,0.998302,NaN,NaN,U12,Walking
22,9.840980,9.814482,9.867058,0.001669,0.016455,209.841434,1005.679930,0.965332,NaN,NaN,U12,Still
23,9.821242,9.765504,9.880349,0.001690,0.031281,124.568661,1006.011000,0.885837,NaN,NaN,U12,Still
44,9.735955,8.653051,10.823420,0.012776,0.363732,99.756594,1008.801098,0.764437,1483.00,NaN,U12,Train
55,9.814477,9.780049,9.852852,0.001625,0.021487,354.465346,1004.094434,0.048799,28966.00,NaN,U12,Still


In [32]:
# user = u12 & u3 as test
dataset_test = dataset.loc[dataset['user'] == 'U12']

In [33]:
dataset_test

,accelerometer_mean,accelerometer_min,accelerometer_max,gyroscope_mean,linear_acceleration_mean,orientation_mean,pressure_mean,rotation_vector_mean,step_counter_mean,sound_mean,user,target
0,9.811476,9.758895,9.849411,0.001651,0.020978,354.286933,1004.090261,0.050413,28966.000000,NaN,U12,Still
1,9.939207,7.707437,17.146631,0.036326,0.879220,332.695577,1008.274660,0.999981,NaN,89.200210,U12,Car
2,9.827178,9.804817,9.849262,0.001525,0.034110,74.752653,1005.977540,0.610456,NaN,NaN,U12,Still
3,9.673039,7.659674,12.304298,0.036440,1.488361,373.556039,1006.950440,0.998112,NaN,87.470377,U12,Car
7,9.906011,6.431646,14.202603,0.140902,1.003520,253.102379,1007.783480,0.929187,12992.250000,NaN,U12,Bus
18,10.098494,4.184305,20.965369,1.936651,6.589222,178.274932,NaN,0.998302,NaN,NaN,U12,Walking
22,9.840980,9.814482,9.867058,0.001669,0.016455,209.841434,1005.679930,0.965332,NaN,NaN,U12,Still
23,9.821242,9.765504,9.880349,0.001690,0.031281,124.568661,1006.011000,0.885837,NaN,NaN,U12,Still
44,9.735955,8.653051,10.823420,0.012776,0.363732,99.756594,1008.801098,0.764437,1483.000000,NaN,U12,Train
55,9.814477,9.780049,9.852852,0.001625,0.021487,354.465346,1004.094434,0.048799,28966.000000,NaN,U12,Still


In [87]:
X = dataset.drop(['target'], axis=1)
y = dataset.target

In [88]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [89]:
from catboost import CatBoostClassifier

model = CatBoostClassifier()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

Learning rate set to 0.084867
0:	learn: 1.5007359	total: 56.2ms	remaining: 56.1s
1:	learn: 1.4151800	total: 161ms	remaining: 1m 20s
2:	learn: 1.3409239	total: 232ms	remaining: 1m 17s
3:	learn: 1.2803894	total: 364ms	remaining: 1m 30s
4:	learn: 1.2194105	total: 495ms	remaining: 1m 38s
5:	learn: 1.1687513	total: 571ms	remaining: 1m 34s
6:	learn: 1.1188414	total: 766ms	remaining: 1m 48s
7:	learn: 1.0751107	total: 850ms	remaining: 1m 45s
8:	learn: 1.0354324	total: 986ms	remaining: 1m 48s
9:	learn: 1.0036015	total: 1.09s	remaining: 1m 48s
10:	learn: 0.9720556	total: 1.26s	remaining: 1m 53s
11:	learn: 0.9437802	total: 1.35s	remaining: 1m 51s
12:	learn: 0.9179388	total: 1.48s	remaining: 1m 52s
13:	learn: 0.8953028	total: 1.62s	remaining: 1m 54s
14:	learn: 0.8720868	total: 1.76s	remaining: 1m 55s
15:	learn: 0.8540845	total: 1.86s	remaining: 1m 54s
16:	learn: 0.8284639	total: 2.05s	remaining: 1m 58s
17:	learn: 0.8070070	total: 2.11s	remaining: 1m 55s
18:	learn: 0.7888918	total: 2.2s	remaining: 

158:	learn: 0.2734561	total: 12s	remaining: 1m 3s
159:	learn: 0.2729611	total: 12.3s	remaining: 1m 4s
160:	learn: 0.2722404	total: 12.4s	remaining: 1m 4s
161:	learn: 0.2714554	total: 12.5s	remaining: 1m 4s
162:	learn: 0.2701752	total: 12.5s	remaining: 1m 4s
163:	learn: 0.2686899	total: 12.6s	remaining: 1m 4s
164:	learn: 0.2681725	total: 12.7s	remaining: 1m 4s
165:	learn: 0.2671639	total: 12.8s	remaining: 1m 4s
166:	learn: 0.2666341	total: 13s	remaining: 1m 4s
167:	learn: 0.2656079	total: 13.1s	remaining: 1m 4s
168:	learn: 0.2648027	total: 13.1s	remaining: 1m 4s
169:	learn: 0.2639528	total: 13.2s	remaining: 1m 4s
170:	learn: 0.2628276	total: 13.3s	remaining: 1m 4s
171:	learn: 0.2619161	total: 13.3s	remaining: 1m 4s
172:	learn: 0.2610268	total: 13.4s	remaining: 1m 4s
173:	learn: 0.2598921	total: 13.5s	remaining: 1m 3s
174:	learn: 0.2591699	total: 13.5s	remaining: 1m 3s
175:	learn: 0.2582233	total: 13.6s	remaining: 1m 3s
176:	learn: 0.2575110	total: 13.6s	remaining: 1m 3s
177:	learn: 0.25

319:	learn: 0.1763268	total: 23s	remaining: 48.8s
320:	learn: 0.1760343	total: 23s	remaining: 48.7s
321:	learn: 0.1757646	total: 23.1s	remaining: 48.6s
322:	learn: 0.1755711	total: 23.2s	remaining: 48.6s
323:	learn: 0.1749246	total: 23.2s	remaining: 48.5s
324:	learn: 0.1743350	total: 23.3s	remaining: 48.4s
325:	learn: 0.1739987	total: 23.4s	remaining: 48.4s
326:	learn: 0.1737326	total: 23.5s	remaining: 48.4s
327:	learn: 0.1732459	total: 23.5s	remaining: 48.2s
328:	learn: 0.1729441	total: 23.6s	remaining: 48.1s
329:	learn: 0.1725437	total: 23.7s	remaining: 48.1s
330:	learn: 0.1722662	total: 23.8s	remaining: 48.1s
331:	learn: 0.1717667	total: 23.8s	remaining: 47.9s
332:	learn: 0.1713386	total: 23.8s	remaining: 47.8s
333:	learn: 0.1709864	total: 23.9s	remaining: 47.6s
334:	learn: 0.1706974	total: 24s	remaining: 47.7s
335:	learn: 0.1702575	total: 24.1s	remaining: 47.6s
336:	learn: 0.1698915	total: 24.1s	remaining: 47.5s
337:	learn: 0.1696955	total: 24.2s	remaining: 47.5s
338:	learn: 0.1690

481:	learn: 0.1315252	total: 34.6s	remaining: 37.2s
482:	learn: 0.1311603	total: 34.6s	remaining: 37.1s
483:	learn: 0.1308888	total: 34.7s	remaining: 37s
484:	learn: 0.1306341	total: 34.8s	remaining: 37s
485:	learn: 0.1303195	total: 34.9s	remaining: 36.9s
486:	learn: 0.1301395	total: 34.9s	remaining: 36.8s
487:	learn: 0.1299651	total: 35.1s	remaining: 36.8s
488:	learn: 0.1297781	total: 35.1s	remaining: 36.7s
489:	learn: 0.1294127	total: 35.2s	remaining: 36.6s
490:	learn: 0.1292455	total: 35.2s	remaining: 36.5s
491:	learn: 0.1290691	total: 35.3s	remaining: 36.4s
492:	learn: 0.1285658	total: 35.4s	remaining: 36.4s
493:	learn: 0.1282312	total: 35.4s	remaining: 36.3s
494:	learn: 0.1280421	total: 35.7s	remaining: 36.4s
495:	learn: 0.1278040	total: 35.8s	remaining: 36.3s
496:	learn: 0.1276995	total: 35.9s	remaining: 36.3s
497:	learn: 0.1274736	total: 35.9s	remaining: 36.2s
498:	learn: 0.1272182	total: 36s	remaining: 36.1s
499:	learn: 0.1270326	total: 36.1s	remaining: 36.1s
500:	learn: 0.1267

642:	learn: 0.1016394	total: 47.6s	remaining: 26.4s
643:	learn: 0.1014064	total: 47.6s	remaining: 26.3s
644:	learn: 0.1012436	total: 47.7s	remaining: 26.3s
645:	learn: 0.1011132	total: 47.8s	remaining: 26.2s
646:	learn: 0.1008350	total: 47.9s	remaining: 26.1s
647:	learn: 0.1006608	total: 48s	remaining: 26.1s
648:	learn: 0.1005258	total: 48s	remaining: 26s
649:	learn: 0.1002797	total: 48.2s	remaining: 25.9s
650:	learn: 0.0999725	total: 48.3s	remaining: 25.9s
651:	learn: 0.0998536	total: 48.4s	remaining: 25.8s
652:	learn: 0.0996992	total: 48.5s	remaining: 25.8s
653:	learn: 0.0994920	total: 48.6s	remaining: 25.7s
654:	learn: 0.0992746	total: 48.7s	remaining: 25.7s
655:	learn: 0.0991735	total: 48.8s	remaining: 25.6s
656:	learn: 0.0989788	total: 48.9s	remaining: 25.5s
657:	learn: 0.0987850	total: 49s	remaining: 25.5s
658:	learn: 0.0986494	total: 49.2s	remaining: 25.4s
659:	learn: 0.0985569	total: 49.2s	remaining: 25.4s
660:	learn: 0.0984492	total: 49.3s	remaining: 25.3s
661:	learn: 0.098354

803:	learn: 0.0814223	total: 1m 3s	remaining: 15.4s
804:	learn: 0.0812774	total: 1m 3s	remaining: 15.3s
805:	learn: 0.0811768	total: 1m 3s	remaining: 15.3s
806:	learn: 0.0810295	total: 1m 3s	remaining: 15.2s
807:	learn: 0.0809748	total: 1m 3s	remaining: 15.1s
808:	learn: 0.0808890	total: 1m 3s	remaining: 15s
809:	learn: 0.0808054	total: 1m 3s	remaining: 14.9s
810:	learn: 0.0807122	total: 1m 3s	remaining: 14.8s
811:	learn: 0.0806355	total: 1m 3s	remaining: 14.8s
812:	learn: 0.0805107	total: 1m 3s	remaining: 14.7s
813:	learn: 0.0803398	total: 1m 3s	remaining: 14.6s
814:	learn: 0.0802409	total: 1m 4s	remaining: 14.5s
815:	learn: 0.0801092	total: 1m 4s	remaining: 14.5s
816:	learn: 0.0800246	total: 1m 4s	remaining: 14.4s
817:	learn: 0.0799046	total: 1m 4s	remaining: 14.3s
818:	learn: 0.0798541	total: 1m 4s	remaining: 14.2s
819:	learn: 0.0798114	total: 1m 4s	remaining: 14.1s
820:	learn: 0.0797230	total: 1m 4s	remaining: 14.1s
821:	learn: 0.0796543	total: 1m 4s	remaining: 14s
822:	learn: 0.07

961:	learn: 0.0675431	total: 1m 15s	remaining: 2.96s
962:	learn: 0.0674746	total: 1m 15s	remaining: 2.88s
963:	learn: 0.0673063	total: 1m 15s	remaining: 2.81s
964:	learn: 0.0672336	total: 1m 15s	remaining: 2.73s
965:	learn: 0.0671540	total: 1m 15s	remaining: 2.65s
966:	learn: 0.0671290	total: 1m 15s	remaining: 2.57s
967:	learn: 0.0670691	total: 1m 15s	remaining: 2.49s
968:	learn: 0.0670276	total: 1m 15s	remaining: 2.41s
969:	learn: 0.0669120	total: 1m 15s	remaining: 2.33s
970:	learn: 0.0668485	total: 1m 15s	remaining: 2.25s
971:	learn: 0.0667731	total: 1m 15s	remaining: 2.17s
972:	learn: 0.0666949	total: 1m 15s	remaining: 2.1s
973:	learn: 0.0666523	total: 1m 15s	remaining: 2.02s
974:	learn: 0.0665265	total: 1m 15s	remaining: 1.94s
975:	learn: 0.0664440	total: 1m 15s	remaining: 1.86s
976:	learn: 0.0663528	total: 1m 15s	remaining: 1.79s
977:	learn: 0.0663194	total: 1m 16s	remaining: 1.71s
978:	learn: 0.0661734	total: 1m 16s	remaining: 1.63s
979:	learn: 0.0660968	total: 1m 16s	remaining: 

In [90]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9172236503856042